In [22]:
import pyspark.sql
from pyspark.sql import *
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate() # on crée la session

data = spark.read.csv("data/demographics_de.csv", header=True, inferSchema=True) # on récupère le csv et on définit les paramètres

data.printSchema() # on affiche le schema

root
 |-- state: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age_group: string (nullable = true)
 |-- population: integer (nullable = true)



In [5]:
data.createOrReplaceTempView("demographics") # on crée une vue temporaire en la nommant
request = spark.sql("select * from demographics")
request.show()  #

+------------------+------+---------+----------+
|             state|gender|age_group|population|
+------------------+------+---------+----------+
|Baden-Wuerttemberg|female|    00-04|    261674|
|Baden-Wuerttemberg|female|    05-14|    490822|
|Baden-Wuerttemberg|female|    15-34|   1293488|
|Baden-Wuerttemberg|female|    35-59|   1919649|
|Baden-Wuerttemberg|female|    60-79|   1182736|
|Baden-Wuerttemberg|female|    80-99|    419471|
|Baden-Wuerttemberg|  male|    00-04|    274882|
|Baden-Wuerttemberg|  male|    05-14|    517387|
|Baden-Wuerttemberg|  male|    15-34|   1423367|
|Baden-Wuerttemberg|  male|    35-59|   1955828|
|Baden-Wuerttemberg|  male|    60-79|   1065669|
|Baden-Wuerttemberg|  male|    80-99|    264560|
|            Bayern|female|    00-04|    306378|
|            Bayern|female|    05-14|    560549|
|            Bayern|female|    15-34|   1507659|
|            Bayern|female|    35-59|   2310659|
|            Bayern|female|    60-79|   1421273|
|            Bayern|

In [6]:
table = spark.table('demographics')
table.show()

+------------------+------+---------+----------+
|             state|gender|age_group|population|
+------------------+------+---------+----------+
|Baden-Wuerttemberg|female|    00-04|    261674|
|Baden-Wuerttemberg|female|    05-14|    490822|
|Baden-Wuerttemberg|female|    15-34|   1293488|
|Baden-Wuerttemberg|female|    35-59|   1919649|
|Baden-Wuerttemberg|female|    60-79|   1182736|
|Baden-Wuerttemberg|female|    80-99|    419471|
|Baden-Wuerttemberg|  male|    00-04|    274882|
|Baden-Wuerttemberg|  male|    05-14|    517387|
|Baden-Wuerttemberg|  male|    15-34|   1423367|
|Baden-Wuerttemberg|  male|    35-59|   1955828|
|Baden-Wuerttemberg|  male|    60-79|   1065669|
|Baden-Wuerttemberg|  male|    80-99|    264560|
|            Bayern|female|    00-04|    306378|
|            Bayern|female|    05-14|    560549|
|            Bayern|female|    15-34|   1507659|
|            Bayern|female|    35-59|   2310659|
|            Bayern|female|    60-79|   1421273|
|            Bayern|

## Exercice 1: Afficher les cinq premières lignes du DataFrame.



In [8]:
table.select('*').limit(5).show()

+------------------+------+---------+----------+
|             state|gender|age_group|population|
+------------------+------+---------+----------+
|Baden-Wuerttemberg|female|    00-04|    261674|
|Baden-Wuerttemberg|female|    05-14|    490822|
|Baden-Wuerttemberg|female|    15-34|   1293488|
|Baden-Wuerttemberg|female|    35-59|   1919649|
|Baden-Wuerttemberg|female|    60-79|   1182736|
+------------------+------+---------+----------+



## Exercice 2: Compter le nombre total de lignes dans le DataFrame.



In [26]:
table.count()

192

## Exercice 3: Trouver le nombre distinct d'états (states) présents dans le DataFrame.



In [15]:
table.select('state').distinct().count()

16

## Exercice 4: Calculer la somme de la population totale.



In [45]:
popTot = table.agg(sum(col('population')))
popTot.show()

+---------------+
|sum(population)|
+---------------+
|       83019213|
+---------------+



## Exercice 5: Afficher la population maximale par groupe d'âge (age_group).



```sql
select sum(population), age_group
from table
group by age_group;
```

In [30]:
table.select('population','age_group').groupby('age_group').agg(max(col('population'))).show()

+---------+---------------+
|age_group|max(population)|
+---------+---------------+
|    80-99|         729148|
|    05-14|         832251|
|    35-59|        3147565|
|    60-79|        2009976|
|    00-04|         440708|
|    15-34|        2188068|
+---------+---------------+



## Exercice 6: Filtrer les lignes du DataFrame pour n'inclure que les états (states) dont la population dépasse 1 million.



```sql
select states, population
from table
where population > 1000000;
```

In [35]:
table.select('state','population').filter(col('population') > 1000000).show()

+-------------------+----------+
|              state|population|
+-------------------+----------+
| Baden-Wuerttemberg|   1293488|
| Baden-Wuerttemberg|   1919649|
| Baden-Wuerttemberg|   1182736|
| Baden-Wuerttemberg|   1423367|
| Baden-Wuerttemberg|   1955828|
| Baden-Wuerttemberg|   1065669|
|             Bayern|   1507659|
|             Bayern|   2310659|
|             Bayern|   1421273|
|             Bayern|   1639500|
|             Bayern|   2348310|
|             Bayern|   1285577|
|             Hessen|   1111053|
|             Hessen|   1111560|
|      Niedersachsen|   1392647|
|      Niedersachsen|   1391992|
|Nordrhein-Westfalen|   2047533|
|Nordrhein-Westfalen|   3147565|
|Nordrhein-Westfalen|   2009976|
|Nordrhein-Westfalen|   2188068|
+-------------------+----------+
only showing top 20 rows



## Exercice 7: Calculer la moyenne de la population pour chaque groupe d'âge (age_group).



```sql
select age_group, avg(population)
from table
group by age_group;
```

In [36]:
table.select('*').groupby('age_group').agg(avg(col('population'))).show()

+---------+---------------+
|age_group|avg(population)|
+---------+---------------+
|    80-99|    168409.5625|
|    05-14|    230138.0625|
|    35-59|   910557.46875|
|    60-79|     562135.625|
|    00-04|   122699.90625|
|    15-34|   600409.78125|
+---------+---------------+



## Exercice 8: Trier le DataFrame par ordre décroissant de population.



```sql
select *
from table
order by population desc;
```

In [39]:
table.select('*').sort(col('population').desc()).show()

+-------------------+------+---------+----------+
|              state|gender|age_group|population|
+-------------------+------+---------+----------+
|Nordrhein-Westfalen|female|    35-59|   3147565|
|Nordrhein-Westfalen|  male|    35-59|   3127984|
|             Bayern|  male|    35-59|   2348310|
|             Bayern|female|    35-59|   2310659|
|Nordrhein-Westfalen|  male|    15-34|   2188068|
|Nordrhein-Westfalen|female|    15-34|   2047533|
|Nordrhein-Westfalen|female|    60-79|   2009976|
| Baden-Wuerttemberg|  male|    35-59|   1955828|
| Baden-Wuerttemberg|female|    35-59|   1919649|
|Nordrhein-Westfalen|  male|    60-79|   1775275|
|             Bayern|  male|    15-34|   1639500|
|             Bayern|female|    15-34|   1507659|
| Baden-Wuerttemberg|  male|    15-34|   1423367|
|             Bayern|female|    60-79|   1421273|
|      Niedersachsen|female|    35-59|   1392647|
|      Niedersachsen|  male|    35-59|   1391992|
| Baden-Wuerttemberg|female|    15-34|   1293488|


## Exercice 9: Sélectionner les lignes du DataFrame où l'âge (age_group) est "15-34" et le sexe (gender) est "female".



```sql
select *
from table
where age_group = "18-24" and gender = "F";
```

In [43]:
table.filter((col('age_group') == '15-34') & (col('gender') == 'female')).show()

+--------------------+------+---------+----------+
|               state|gender|age_group|population|
+--------------------+------+---------+----------+
|  Baden-Wuerttemberg|female|    15-34|   1293488|
|              Bayern|female|    15-34|   1507659|
|              Berlin|female|    15-34|    477479|
|         Brandenburg|female|    15-34|    213065|
|              Bremen|female|    15-34|     83990|
|             Hamburg|female|    15-34|    246744|
|              Hessen|female|    15-34|    715254|
|Mecklenburg-Vorpo...|female|    15-34|    146826|
|       Niedersachsen|female|    15-34|    875509|
| Nordrhein-Westfalen|female|    15-34|   2047533|
|     Rheinland-Pfalz|female|    15-34|    448341|
|            Saarland|female|    15-34|    103235|
|             Sachsen|female|    15-34|    386154|
|      Sachsen-Anhalt|female|    15-34|    193271|
|  Schleswig-Holstein|female|    15-34|    305125|
|          Thueringen|female|    15-34|    191712|
+--------------------+------+--

## Exercice 10: Ajouter une nouvelle colonne "population_percentage" qui représente la proportion de la population par rapport à la population totale.



In [55]:
addCol = table.withColumn('population_percentage', round(col('population') / popTot.collect()[0][0]*100, 2))
addCol.show()

+------------------+------+---------+----------+---------------------+
|             state|gender|age_group|population|population_percentage|
+------------------+------+---------+----------+---------------------+
|Baden-Wuerttemberg|female|    00-04|    261674|                 0.32|
|Baden-Wuerttemberg|female|    05-14|    490822|                 0.59|
|Baden-Wuerttemberg|female|    15-34|   1293488|                 1.56|
|Baden-Wuerttemberg|female|    35-59|   1919649|                 2.31|
|Baden-Wuerttemberg|female|    60-79|   1182736|                 1.42|
|Baden-Wuerttemberg|female|    80-99|    419471|                 0.51|
|Baden-Wuerttemberg|  male|    00-04|    274882|                 0.33|
|Baden-Wuerttemberg|  male|    05-14|    517387|                 0.62|
|Baden-Wuerttemberg|  male|    15-34|   1423367|                 1.71|
|Baden-Wuerttemberg|  male|    35-59|   1955828|                 2.36|
|Baden-Wuerttemberg|  male|    60-79|   1065669|                 1.28|
|Baden